To support the impact score for proteins on pathways, we also analyzed a scRNA-seq dataset downloaded from the tabular project. We did a sysetmatic correlation analysis using BigScale, a R package () using R. The generated correlation data was exported into a file and then loaded into this notebook for further analysis. The basic step is similar to the NLP analysis. Therefore, this notebook is placed in this folder for easy maintenance.

In [ ]:
from IPython.display import display, HTML
# Make sure all width is used to better take the screen space
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Reuse functions used for NLP
%run ./TopicModelingResultAnalyzer.py
%run ./ReactomePathwayTopicModeling.py
# Make sure this folder is right
dir_name = '/Volumes/ssd/results/reactome-idg/fi-network-ml/impact_analysis/scRNASeq'

In [ ]:
cor_file_name = dir_name + '/' + 'tabula_blood_corr_recursive.csv'
# The correlation is pair-wise and the file is big. The loading is a quite slow process.
cor_df = pd.read_csv(cor_file_name, index_col=0)

In [ ]:
print(cor_df.shape)
cor_df.head()

In [ ]:
# Need to fix the column names. An issue from R
cor_df.columns = cor_df.columns.map(lambda c : c.replace('.', '-'))
cor_df

In [ ]:
cor_df.stack().plot.hist(bins=1000)
# The distribution is quite positive with the peak around 0.50. This is a little bit unexpected!

In [ ]:
cor_df_statcked = cor_df.stack()

In [ ]:
cor_df_statcked.sort_values(inplace=True, ascending=False)

In [ ]:
size = cor_df_statcked.shape[0]
print(size)
print(np.NAN)
cor_df_statcked[int(size * 0.001 + cor_df.shape[0])]

In [ ]:
35803 + 16000 + 6098

In [ ]:
# The following code was modified from Patrick's original jupyter notebook code
gmt_file = dir_name + '/../' + 'ReactomePathwayGenes_Ver_77.txt'
gmt_df = pd.read_csv(gmt_file, sep='\t', header=None)
gmt_df.columns = ('DB_ID', 'genes')
gmt_df['pathway_genes'] = gmt_df['genes'].map(lambda genes : genes.split(','))
gmt_df.set_index('DB_ID', drop=True, inplace=True)
print(gmt_df.shape)
gmt_df.head()

In [ ]:
gmt_df['pathway_genes_in_cor'] = gmt_df['pathway_genes'].map(lambda genes : cor_df.index[cor_df.index.isin(genes)].to_list())
gmt_df.head()

In [ ]:
impact_result_df = load_impact_results()
print(impact_result_df.shape)
impact_result_df.head()

In [ ]:
# Functions use to calculate median and mean values of correlations for pathway genes.
counter = 0
def calculate_cor_summary(pathway_id: int,
                          gene: str,
                          gmt_df: pd.DataFrame,
                          cor_df: pd.DataFrame,
                          func) -> float:
    global counter
    counter = counter + 1
    # Get pathway genes
    # Need to copy the list to avoid change the original list in the dataframe
    pathway_genes = [*gmt_df.loc[pathway_id]['pathway_genes_in_cor']]
    # Remove gene itself if it is there
    if gene in pathway_genes:
        pathway_genes.remove(gene)
    # Get the correlation values for gene
    gene_cor = cor_df.loc[gene]
    # The above returns a series indexed by gene names
    pathway_gene_cor = gene_cor[pathway_genes]
    # Pick pairs having at least three values
    if pathway_gene_cor.size < 3:
        # print('Not enough genes for {} vs {}.'.format(pathway_id, gene))
        return np.NaN
    rtn = func(np.abs(pathway_gene_cor))
    if counter % 1000 == 0:
        print("Total running: {}.".format(counter))
    return rtn

def calculate_row_summary(row, func) -> float:
    return calculate_cor_summary(row.DBID,
                                 row.Gene,
                                 gmt_df,
                                 cor_df,
                                 func)

value = calculate_cor_summary(416476, 'NOC2L', gmt_df, cor_df, func=np.mean)
print("Mean: {}".format(value))
# Based on https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html
# %load_ext line_profiler
# %lprun -f calculate_cor_summary calculate_cor_summary(416476, 'NOC2L', gmt_df, cor_df, func=np.mean)

In [ ]:
#Make sure genes in impact_results have correlation values
print('Before filtering: {}'.format(impact_result_df.shape))
selected_rows = impact_result_df['Gene'].isin(cor_df.index)
impact_result_df_selected = impact_result_df[selected_rows]
print('After filtering: {}'.format(impact_result_df_selected.shape))
impact_result_df_selected.head()
# add two columns
# Two slow steps for multiple calculations
# For quick check
# impact_result_df_selected = impact_result_df_selected.iloc[:10000, ]
counter = 0
impact_result_df_selected['Median_Cor'] = impact_result_df_selected.apply(lambda row : calculate_row_summary(row, np.median), axis=1)
counter = 0
impact_result_df_selected['Mean_Cor'] = impact_result_df_selected.apply(lambda row : calculate_row_summary(row, np.mean), axis=1)
impact_result_df_selected.head()

In [ ]:
# Save the output to avoid running it again
file_name = dir_name + '/' + 'impact_result_df_selected_081922.csv'
impact_result_df_selected.to_csv(file_name)

In [ ]:
# Re-load the file
file_name = dir_name + '/' + 'impact_result_df_selected_081922.csv'
impact_result_df_selected = pd.read_csv(file_name)

In [ ]:
from scipy import stats
import numpy as np
etf1_result = impact_result_df_selected[impact_result_df_selected.Gene == 'ETF1']
print(etf1_result.head())
stats.pearsonr(etf1_result.Median_Cor, -np.log10(etf1_result.FDR))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_figwidth(30)
fig.set_figheight(16)
sns.histplot(impact_result_df_selected.Median_Cor, ax = axes[0][0])
sns.histplot(impact_result_df_selected.Mean_Cor, ax = axes[0][1])
sns.histplot(-np.log10(impact_result_df_selected.FDR), ax = axes[1][0])
g = sns.histplot(impact_result_df_selected.Average_Activation, ax = axes[1][1])
g.set_ylim(0, 25000)

In [ ]:
# Calculate correlation
# %pip install -U ipykernel
# print(impact_result_df_selected.columns)
def calculate_correlation(impact_col_name: str = 'FDR',
                          cor_col_name: str = 'Median_Cor',
                          impact_result_df_selected: pd.DataFrame = impact_result_df_selected):
    df_cor_fdr_cor = pd.DataFrame(columns=['Gene', 'Pearson_Cor', 'Peason_P_Value', 'Spearm_Cor', 'Spearm_P_Value', 'length'])
    new_index = 0
    for gene in impact_result_df_selected.Gene.unique():
        # if gene != 'RPL11':
        #     continue
        which_rows = impact_result_df_selected.Gene == gene
        pathway_median_gene = impact_result_df_selected[which_rows]
        # print(pathway_median_gene.head())
        pathway_median_gene = impact_result_df_selected[which_rows][['DBID', cor_col_name]]
        pathway2median = dict()
        for index, row in pathway_median_gene.iterrows():
            if np.isnan(row[cor_col_name]):
                continue
            pathway2median[int(row['DBID'])] = row[cor_col_name]
        gene_impact_df = impact_result_df_selected[impact_result_df_selected.Gene == gene]
        results = calculate_cor_impact_cosine(pathway2median, 
                                              gene_impact_df,
                                              col_name = impact_col_name,
                                              use_pathway_name=False)
        # print(results)
        if results is None:
            continue
        df_cor_fdr_cor.loc[new_index] = [gene, results[0][0], results[0][1], results[1][0], results[1][1], results[2]]
        new_index = new_index + 1
        if new_index % 1000 == 0:
            print(new_index)
        # if new_index == 5:
        #     break    
    return df_cor_fdr_cor


In [ ]:
# Correlation between FDR and median_cor
df_cor_fdr_cor = calculate_correlation(impact_col_name='FDR')
df_cor_fdr_cor.head()
# Save the output of the above
#file_name = dir_name + '/df_cor_fdr_median_cor_081922.csv'
#df_cor_fdr_cor.to_csv(file_name)

In [ ]:
# Correlation between Average Activation and meidan_cor
df_cor_activation_cor = calculate_correlation(impact_col_name='Average_Activation')
print(df_cor_activation_cor.head())
file_name = dir_name + '/df_cor_average_activation_median_cor_081922.csv'
df_cor_activation_cor.to_csv(file_name)

In [ ]:
# Correlation between Average inhibition and meidan_cor
df_cor_inhibition_cor = calculate_correlation(impact_col_name='Average_Inhibition')
print(df_cor_inhibition_cor.head())
file_name = dir_name + '/df_cor_average_inhibition_median_cor_081922.csv'
df_cor_inhibition_cor.to_csv(file_name)

In [ ]:
# import pandas
file_name = dir_name + '//df_cor_average_inhibition_median_cor_081922.csv'
df_cor_inhibition_cor = pandas.read_csv(file_name)
fig, axes = plt.subplots(nrows=1, ncols=3)
fig.set_figwidth(30)
fig.set_figheight(10)
sns.scatterplot(x = df_cor_fdr_cor.Pearson_Cor, y = -np.log10(df_cor_fdr_cor.Peason_P_Value), ax = axes[0], size=8)
sns.scatterplot(x = df_cor_activation_cor.Pearson_Cor, y = -np.log10(df_cor_activation_cor.Peason_P_Value), ax = axes[1], size=8)
sns.scatterplot(x = df_cor_inhibition_cor.Pearson_Cor, y = -np.log10(df_cor_inhibition_cor.Peason_P_Value), ax = axes[2], size=8)

Try to follow the original approach used in the BigScale paper about building gene regulatory network by choosing the top 0.1% of correlations (not absolute values). Performed an enrichment analysis for each gene in the filtered network in Java. The following is correlation analysis between this enrichment scores and previous enrichment scores.

In [ ]:
import pandas as pd
file_name = dir_name + "/../impact_analysis_092121_with_enrichment_092921_with_scRNASeq_082322.txt"
impact_result_df_with_bigscale = pd.read_csv(file_name, sep = '\t')
impact_result_df_with_bigscale.head()

In [ ]:
impact_result_df_with_bigscale['BigScale_FDR_Log10'] = impact_result_df_with_bigscale.BigScale_FDR.map(lambda f : -np.log10(f))
print(impact_result_df_with_bigscale.head())
df_cor_fdr_bigscale = calculate_correlation(cor_col_name='BigScale_FDR_Log10', impact_result_df_selected=impact_result_df_with_bigscale)
file_name = dir_name + '/df_cor_fdr_bigscale_082322.csv'
df_cor_fdr_bigscale.to_csv(file_name)

In [ ]:
df_cor_average_activation_bigscale = calculate_correlation(cor_col_name='BigScale_FDR_Log10', 
                                                           impact_col_name='Average_Activation',
                                                           impact_result_df_selected=impact_result_df_with_bigscale)
file_name = dir_name + '/df_cor_average_activation_bigscale_082322.csv'
df_cor_average_activation_bigscale.to_csv(file_name)
df_cor_average_inhibition_bigscale = calculate_correlation(cor_col_name='BigScale_FDR_Log10', 
                                                           impact_col_name='Average_Inhibition',
                                                           impact_result_df_selected=impact_result_df_with_bigscale)
file_name = dir_name + '/df_cor_average_inhibition_bigscale_082322.csv'
df_cor_average_inhibition_bigscale.to_csv(file_name)

In [ ]:
# Plots
fig, axes = plt.subplots(nrows=1, ncols=3)
fig.set_figwidth(30)
fig.set_figheight(10)
sns.scatterplot(x = df_cor_fdr_bigscale.Pearson_Cor, y = -np.log10(df_cor_fdr_bigscale.Peason_P_Value), ax = axes[0], size=8)
sns.scatterplot(x = df_cor_average_activation_bigscale.Pearson_Cor, y = -np.log10(df_cor_average_activation_bigscale.Peason_P_Value), ax = axes[1], size=8)
# Typo in the above
df_cor_average_inhibition_bigscale = df_cor_inhibition_activation_bigscale
sns.scatterplot(x = df_cor_average_inhibition_bigscale.Pearson_Cor, y = -np.log10(df_cor_average_inhibition_bigscale.Peason_P_Value), ax = axes[2], size=8)